In [9]:
import os

import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner

import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##  Configure SageMaker environment

In [10]:
# set SageMaker role
os.environ['AWS_PROFILE']='sagemaker'
os.environ['AWS_DEFAULT_REGION']='eu-central-1'

In [11]:
session = sagemaker.Session()
bucket = session.default_bucket()
print(bucket)
prefix = "sagemaker/ml-capistone-project"
role = sagemaker.get_execution_role()

sagemaker-eu-central-1-292065287762


## Prepare data

In [12]:
# Load data
stock_list = ['NFLX', 'EPAM', 'AAPL']

end = datetime.now()
start = datetime(end.year - 5, end.month, end.day)

for stock in stock_list:
    globals()[stock] = yf.download(stock, start, end)

stock_data = [NFLX, EPAM, AAPL]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [13]:
# set data
stock_df = EPAM
stock_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-10-16,90.959999,91.699997,90.300003,91.150002,91.150002,262400
2017-10-17,91.099998,91.970001,90.650002,90.949997,90.949997,172300
2017-10-18,91.000000,91.139999,90.540001,90.540001,90.540001,161700
2017-10-19,90.349998,90.980003,89.820000,90.540001,90.540001,194800
2017-10-20,91.139999,91.459999,91.110001,91.410004,91.410004,275200


In [14]:
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1259 entries, 2017-10-16 to 2022-10-14
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1259 non-null   float64
 1   High       1259 non-null   float64
 2   Low        1259 non-null   float64
 3   Close      1259 non-null   float64
 4   Adj Close  1259 non-null   float64
 5   Volume     1259 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 68.9 KB


In [15]:
# save data
data_folder='data'
os.makedirs(data_folder, exist_ok=True)  
stock_df.to_csv(data_folder + '/stock.csv') 

In [16]:
# upload stock data to S3
inputs = session.upload_data(path=data_folder, bucket=bucket, key_prefix=prefix)
print("S3 path: {}".format(inputs))

S3 path: s3://sagemaker-eu-central-1-292065287762/sagemaker/ml-capistone-project


## Hyperparameter tunner

### Find best CPU otimized instance for training

In [95]:
use_spot_instances = True
spot_max_wait = 5000
hyperparameters = {
    'epochs': 600,
    'learning_rate': 0.00001,
    'feature_columns': '"Adj Close"' # for multiple values use: '"Adj Close" "Volume"'
}

In [98]:
# Optimal instances for current paylad.
# CPU instantce ml.c5.2xlarge
# GPU instance ml.g4dn.xlarge
estimator = PyTorch(
    entry_point='hpo.py',
    source_dir='src',
    base_job_name='stock-predictor',
    role=role,
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    framework_version='1.12',
    py_version='py38',
    hyperparameters=hyperparameters,
    use_spot_instances=use_spot_instances,
    max_wait=2500,
    max_run=2500
)

In [99]:
estimator.fit({'data': inputs}, wait=True)

2022-10-16 19:56:57 Starting - Starting the training job...
2022-10-16 19:57:13 Starting - Preparing the instances for trainingProfilerReport-1665950217: InProgress
......
2022-10-16 19:58:30 Downloading - Downloading input data...
2022-10-16 19:58:46 Training - Downloading the training image.............................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-10-16 20:03:45,669 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-10-16 20:03:45,695 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-10-16 20:03:45,703 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-10-16 20:03:46,320 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "data": "/opt/ml/input/data/data"
    },
    "current_host": 

### Find best learning rate

In [47]:
hyperparameters = {
    'epochs': 600,
    'feature_columns': '"Adj Close"' # for multiple values use: '"Adj Close" "Volume"'
}

In [64]:
estimator = PyTorch(
    entry_point='hpo.py',
    source_dir='src',
    base_job_name='stock-predictor',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version='1.12',
    py_version='py38',
    hyperparameters=hyperparameters,
)

In [65]:
objective_metric_name = "Test Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Test Loss", "Regex": "Train loss: ([0-9\\.]+)"}]

In [66]:
hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.00005, 0.0005),
}

In [51]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=9,
    max_parallel_jobs=3,
    objective_type=objective_type
)

In [52]:
tuner.fit({'data': inputs}, wait=True)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.....................................................................................................................................................................................................................................................................................................................................................................!


## Stock estimator

In [24]:
hyperparameters = {
    'epochs': 3,
    'learning_rate': 0.00001,
    'feature_columns': '"Adj Close"' # for multiple values use: '"Adj Close" "Volume"'
}
hyperparameters

{'learning_rate': 1e-05, 'feature_columns': '"Adj Close"'}

In [25]:
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

In [26]:
hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "1",
        "eval.save_interval": "1"
    }
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=1)
)

In [27]:
estimator = PyTorch(
    entry_point='hpo.py',
    source_dir='src',
    base_job_name='stock-predictor',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version='1.12',
    py_version='py38',
    hyperparameters=hyperparameters,
    ## Debugger and Profiler parameters
    rules = rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
)

In [28]:
estimator.fit({'data': inputs}, wait=True)

2022-10-14 22:45:48 Starting - Starting the training job...VanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
ProfilerReport: InProgress
......
2022-10-14 22:47:07 Starting - Preparing the instances for training......
2022-10-14 22:48:21 Downloading - Downloading input data......
2022-10-14 22:49:21 Training - Downloading the training image......
2022-10-14 22:50:22 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-10-14 22:50:15,337 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-10-14 22:50:15,339 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-10-14 22:50:15,352 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-10-14 22:50:15,360 sagemaker_pytorch_container.trai